In [1]:
import os
import sys
import json
import torch
import torchvision
from tqdm import tqdm_notebook as tqdm
from transformers import BertModel, BertTokenizer

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

I0317 01:51:51.845125 140574716786496 file_utils.py:39] PyTorch version 1.1.0 available.


'/work'

In [2]:
sys.path.append(w_dir+'/fgc_support_retri')

In [3]:
from fgc_support_retri.ser_extractor import *
from fgc_support_retri.utils import read_fgc, read_hotpot, json_load, json_write
from fgc_support_retri import config
from evaluation.eval import eval_sp_fgc, eval_fgc_atype

In [4]:
extractor = EntityMatch_extractor()

I0317 01:51:58.895591 140574716786496 tokenization_utils.py:375] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
I0317 01:51:59.861984 140574716786496 configuration_utils.py:152] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.3767c74c8ed285531d04153fe84a0791672aff52f7249b27df341dbce09b8305
I0317 01:51:59.864810 140574716786496 configuration_utils.py:169] Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_r

In [5]:
data = json_load(config.FGC_TEST)

In [6]:
all_sp_predictions = []
all_atype_predictions = []
for d in tqdm(data):
    for q in d['QUESTIONS']:
        sp_preds, _, sp_scores = extractor.predict(q, d)
        all_sp_predictions.append(sp_preds)
        q['sp'] = sp_preds
        q['sp_scores'] = sp_scores
#         except Exception as e:
#             print(e)
#             print(d)
#             break
#     all_atype_predictions += atypes

100%|██████████| 193/193 [02:59<00:00,  1.55s/it]


In [7]:
all_items = []
for d in tqdm(data):
    for q in d['QUESTIONS']:
        all_items.append(q['SHINT'])

100%|██████████| 193/193 [00:00<00:00, 73994.58it/s]


In [8]:
metrics = eval_sp_fgc(all_items, all_sp_predictions)
# atype_accuracy = eval_fgc_atype(all_items, all_atype_predictions)

{'sp_em': 0.17616580310880828, 'sp_prec': 0.616627600954026, 'sp_recall': 0.5957685664939552, 'sp_f1': 0.5605767577270172}


# Write output

In [9]:
config.FGC_TEST

PosixPath('/work/fgc_support_retri/data/FGC/FGC_release_1.7.7/FGC_release_all_test.json')

In [16]:
json_write(data, 'FGC_release_all_test.json')

In [10]:
qids = []
questions = []
t_sp = []
p_sp = []

q_idx = 0 
for document in data:
    for question in document['QUESTIONS']:
        question['sp_predict'] = all_sp_predictions[q_idx]
        q_idx +=1
        
        qids.append(question['QID'])
        questions.append(question['QTEXT_CN'])
        t_sp_text = []
        p_sp_text = []
        for idx in question['SHINT']:
            t_sp_text.append(document['SENTS'][idx]['text'])
        for idx in question['sp_predict']:
            p_sp_text.append(document['SENTS'][idx]['text'])
        t_sp.append(t_sp_text)
        p_sp.append(p_sp_text)

In [11]:
import pandas as pd

pred_df = pd.DataFrame(index=qids)
pred_df['question'] = questions
pred_df['t_sp'] = t_sp
pred_df['p_sp'] = p_sp

In [12]:
pred_df.head(5)

,question,t_sp,p_sp
D003Q01,上官均是否赞成王安石的新政改革?,[因发现上官均的策论有诋毁王安石变法的情况，],[便改上官均为第二名。]
D003Q02,苏东坡因为什么事件入狱几乎要死在狱中?,"[因乌台诗案入狱，, 几死，因为写文章向朝廷诀别，]","[因乌台诗案入狱，, 神宗动心，苏轼终免一死，]"
D003Q03,在「乌台诗案」中，有哪些人出面救了苏东坡？,"[几死，因为写文章向朝廷诀别，, 太皇太后曹氏、王安礼等人出面力挽，, 神宗动心，苏轼终免一死，]","[苏轼参加救灾。, 太皇太后曹氏、王安礼等人出面力挽，]"
D003Q04,苏东坡因「乌台诗案」被贬到什么地方？,"[因乌台诗案入狱，, 神宗动心，苏轼终免一死，, 翌年被贬至黄州（今湖北省黄冈市），]","[因乌台诗案入狱，, 贬谪为「检校尚书水部员外郎黄州团练副使本州安置」，, 翌年被贬至黄州（..."
D003Q05,宋神宗熙宁十月，黄河溃堤时，苏东坡人在哪里？,"[熙宁十年四月，, 赴任徐州，是年七月七日，, 黄河决口，水困徐州，, 苏轼参加救灾。]","[熙宁三年（1070年），, 熙宁十年四月，]"


In [13]:
pred_df.to_csv('em_pred.csv')

In [25]:
atype_accuracy

0.8798076923076923

In [ ]:
for d in tqdm(data):
    item = extractor.get_item(d)
    items= [i for i in item]
    for q_id, item in enumerate(items):
        score_list = extractor.predict_score(item)
        d['QUESTIONS'][q_id]['s_score'] = score_list

In [12]:
json_write(data, str(config.RESULT_PATH/"prediction"/"FGC_3_weeks_sup.json"))

In [6]:
data = json_load(config.DATA_ROOT/"FGC"/"FGC_3_weeks.json")

In [11]:
data[0]

{'DID': 'D001',
 'DTEXT': '蘇軾（1037年1月8日－1101年8月24日），眉州眉山（今四川省眉山市）人，北宋時著名的文學家、政治家、藝術家、醫學家。字子瞻，一字和仲，號東坡居士、鐵冠道人。嘉佑二年進士，累官至端明殿學士兼翰林學士，禮部尚書。南宋理學方熾時，加賜諡號文忠，複追贈太師。有《東坡先生大全集》及《東坡樂府》詞集傳世，宋人王宗稷收其作品，編有《蘇文忠公全集》。\n其散文、詩、詞、賦均有成就，且善書法和繪畫，是文學藝術史上的通才，也是公認韻文散文造詣皆比較傑出的大家。蘇軾的散文為唐宋四家（韓愈、柳宗元、歐蘇）之末，與唐代的古文運動發起者韓愈並稱為「韓潮蘇海」，也與歐陽修並稱「歐蘇」；更與父親蘇洵、弟蘇轍合稱「三蘇」，父子三人，同列唐宋八大家。蘇軾之詩與黃庭堅並稱「蘇黃」，又與陸游並稱「蘇陸」；其詞「以詩入詞」，首開詞壇「豪放」一派，振作了晚唐、五代以來綺靡的西崑體餘風。後世與南宋辛棄疾並稱「蘇辛」，惟蘇軾故作豪放，其實清朗；其賦亦頗有名氣，最知名者為貶謫期間借題發揮寫的前後《赤壁賦》。宋代每逢科考常出現其文命題之考試，故當時學者曰：「蘇文熟，喫羊肉、蘇文生，嚼菜羹」。藝術方面，書法名列「蘇、黃、米、蔡」北宋四大書法家（宋四家）之首；其畫則開創了湖州畫派；並在題畫文學史上佔有舉足輕重的地位。',
 'QUESTIONS': [{'QID': 'D001Q09',
   'QTYPE': '基礎題',
   'QTEXT': '辛棄疾是哪一個朝代的人？',
   'ANSWER': [{'ATEXT': '南宋', 'ATEXT_CN': '南宋'}],
   'QTEXT_CN': '辛弃疾是哪一个朝代的人？',
   'QTEXT_RE': '辛弃疾是哪一个朝代的人？',
   'REFINED': 0.9,
   'AMODE': 'Single-Span-Extraction',
   'ATYPE': 'Date-Duration',
   's_score': [0.18831099569797516,
    0.00017647468484938145,
    0.0009485816117376089,
    0.00013792241225019097,
    8.458553929813206e-05,
  

# DRCD

In [7]:
files = os.listdir(config.DATA_ROOT/"release_1.7"/"DRCD")
files = [f for f in files if f.endswith(".json")]

for f in files:
    data = json_load(config.DATA_ROOT/"release_1.7"/"DRCD"/f)
    for d in tqdm(data):
        item = extractor.get_item(d)
        items= [i for i in item]
        for q_id, item in enumerate(items):
            score_list = extractor.predict_score(item)
            d['QUESTIONS'][q_id]['s_score'] = score_list
    json_write(data, str(config.RESULT_PATH/"prediction"/(f[:-5]+"_sup.json")))

 42%|████▏     | 524/1254 [00:27<01:13,  9.95it/s]

tokenized all > 511 id:3_445-1


 85%|████████▍ | 1064/1254 [01:04<00:17, 10.94it/s]

tokenized all > 511 id:7_21053-1


 61%|██████▏   | 491/800 [00:08<00:05, 53.53it/s]

...

 54%|█████▎    | 428/800 [00:07<00:06, 55.73it/s]

tokenized all > 511 id:2_2358-1


 88%|████████▊ | 704/800 [00:12<00:01, 53.83it/s]

tokenized all > 511 id:5_1476-1
tokenized all > 511 id:5_1477-1


100%|██████████| 711/711 [01:22<00:00,  8.60it/s]
